## Set up

In [ ]:
import ee
from ee import batch
ee.Authenticate()
ee.Initialize()

## Functions for dataset

In [ ]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4'])
  newImage = image.addBands(ndvi.rename('NDVI')).float()
  return newImage

def query_collection(aoi, start_string, finish_string, season):
  if season == "dry":
    month_filter = ee.Filter.calendarRange(6, 10, 'month')
  elif season == 'wet':
    month_filter = ee.Filter.calendarRange(11, 5, 'month')

  start = ee.Date(start_string)
  finish = ee.Date(finish_string)
  landsat8 = ee.ImageCollection(
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    # Filter time interval
    .filterDate(start, finish)
    # Filter AOI
    .filterBounds(aoi))
  return landsat8.filter(month_filter)

def maskl8sr(image):
  # Bit 0 - Fill, 0
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  image = image.addBands(opticalBands, None, True).addBands(thermalBands, None, True).updateMask(qaMask).updateMask(saturationMask)

  return image

## Prepare dataset

In [ ]:
# This time interval is chosen based on FAO crop calendar
## Long term
start_string = '2013-06-01'
finish_string = '2019-05-31'

## Short term
# start_string = '2017-06-01'
# finish_string = '2019-05-31'

# Get names of tiles
aoi = ee.FeatureCollection("projects/ee-pinotsung/assets/tanzania")

# DRY SEASON
# query image collection
l8_sr_dry = query_collection(aoi, start_string, finish_string, "dry")

# apply cloud mask and add NDVI
l8_sr_dry = l8_sr_dry.map(maskl8sr).map(addNDVI)
print(l8_sr_dry.size().getInfo())

# WET SEASON
# query image collection
l8_sr_wet = query_collection(aoi, start_string, finish_string, "wet")

# apply cloud mask and add NDVI
l8_sr_wet = l8_sr_wet.map(maskl8sr).map(addNDVI)
print(l8_sr_wet.size().getInfo())

# ALL SEASON
l8_sr = l8_sr_wet.merge(l8_sr_dry)

1937
2392


## Calculate the mean NDVI

In [ ]:
# l8_ndvi_dry = l8_sr_dry.select("NDVI").mean().multiply(100000).toInt()
l8_ndvi_dry = l8_sr_dry.select("NDVI").mean()
l8_ndvi_wet = l8_sr_wet.select("NDVI").mean()

## Calculate mean of annual standard deviation of NDVI

In [ ]:
def sd_calc(year):
  subset = l8_sr.filterDate(year[0], year[1])
  return subset.select("NDVI").reduce(ee.Reducer.stdDev())

In [ ]:
years = [['2013-06-01', '2014-05-31'], ['2014-06-01', '2015-05-31'],
         ['2015-06-01', '2016-05-31'], ['2016-06-01', '2017-05-31'],
         ['2017-06-01', '2018-05-31'], ['2018-06-01', '2019-05-31']]
annual_sd = ee.ImageCollection([sd_calc(year) for year in years])
annual_sd = annual_sd.mean()

## Export result

In [ ]:
# Mean NDVI
scale = 30
name_output = 'lansat8_sr_13_19_{}_NDVI_mean_{}m'.format("dry", scale)
print(name_output)
task = ee.batch.Export.image.toDrive(**{
    'image': l8_ndvi_dry,
    'region': aoi.geometry().buffer(30),
    'description': name_output,
    'folder':'landsat8_ndvi',
    'scale': scale,
    'maxPixels': 1e13
})
task.start()

name_output = 'lansat8_sr_13_19_{}_NDVI_mean_{}m'.format("wet", scale)
print(name_output)
task = ee.batch.Export.image.toDrive(**{
    'image': l8_ndvi_wet,
    'region': aoi.geometry().buffer(30),
    'description': name_output,
    'folder':'landsat8_ndvi',
    'scale': scale,
    'maxPixels': 1e13
})
task.start()

name_output = 'lansat8_sr_13_19_NDVI_mean_sd_{}m'.format(scale)
print(name_output)
task = ee.batch.Export.image.toDrive(**{
    'image': annual_sd,
    'region': aoi.geometry().buffer(30),
    'description': name_output,
    'folder':'landsat8_ndvi',
    'scale': scale,
    'maxPixels': 1e13
})
task.start()

lansat8_sr_13_19_dry_NDVI_mean_30m
lansat8_sr_13_19_wet_NDVI_mean_30m
lansat8_sr_13_19_NDVI_mean_sd_30m


In [ ]:
# Mean NDVI
scale = 1000
name_output = 'lansat8_sr_16_20_{}_NDVI_mean_{}m'.format("dry", scale)
print(name_output)
task = ee.batch.Export.image.toDrive(**{
    'image': l8_ndvi_dry,
    'region': aoi.geometry().buffer(30),
    'description': name_output,
    'folder':'landsat8_ndvi',
    'scale': scale,
    'maxPixels': 1e13
})
task.start()

name_output = 'lansat8_sr_16_20_{}_NDVI_mean_{}m'.format("wet", scale)
print(name_output)
task = ee.batch.Export.image.toDrive(**{
    'image': l8_ndvi_wet,
    'region': aoi.geometry().buffer(30),
    'description': name_output,
    'folder':'landsat8_ndvi',
    'scale': scale,
    'maxPixels': 1e13
})
task.start()

lansat8_sr_16_20_dry_NDVI_mean_1000m
lansat8_sr_16_20_wet_NDVI_mean_1000m


### Cancel tasks if necessary

In [ ]:
!earthengine task cancel all